In [ ]:
import requests
import pandas as pd

In [116]:
def get_all_reviews(appid):
    cursor = '*'
    review_df = pd.DataFrame()
    game_data_df = pd.DataFrame()

    while cursor:
        params = {
            "json": 1,
            "filter": "all",
            "language": 'english',
            "day_range": 365,
            "cursor": cursor,
            "review_type": "all",
            "purchase_type": "all",
            "num_per_page": 100
        }

        review_url = f"https://store.steampowered.com/appreviews/{appid}"
        response = requests.get(review_url, params=params)
        data = response.json()
        if game_data_df.empty:
            game_data_df = pd.DataFrame([data.pop("query_summary")])

        data_df = pd.DataFrame(data.get('reviews'))

        if review_df.empty:
            review_df = data_df
        else:
            review_df = pd.concat([review_df, data_df])

        cursor = urllib.parse.quote(data.get("cursor"))

        if not cursor or len(review_df) >= 1000:
            review_df = review_df.head(1000)
            break
    
    game_data_df.insert(0, 'appid', appid)
    review_df.insert(0, 'appid', appid)

    return game_data_df, review_df

In [ ]:
def get_all_reviews_for_top_1000_games():
    top_1000_games_app_id = pd.read_csv("../../data/game_player_cnt_ranked_top_1k.csv")["appid"]
    all_game_data_df = pd.DataFrame()
    all_reviews_df = pd.DataFrame()

    for index, appid in enumerate(top_1000_games_app_id):
        print(f"Running Appid: {appid}. {round((index/len(top_1000_games_app_id)*100), 0)}% Complete")
        single_game_data_df, single_game_reviews_df = get_all_reviews(appid=appid)
        print(f"\t{len(single_game_reviews_df)} reviews found")

        if all_reviews_df.empty:
            all_game_data_df = single_game_data_df
            all_reviews_df = single_game_reviews_df
        else:
            all_game_data_df = pd.concat([all_game_data_df, single_game_data_df])
            all_reviews_df = pd.concat([all_reviews_df, single_game_reviews_df])

    
    all_game_data_df.to_csv('../../data/top_1000_game_data.csv', index=False)
    all_reviews_df.to_csv('../../data/top_1000_game_reviews.csv', index=False)
    return all_reviews_df

In [ ]:
df = get_all_reviews_for_top_1000_games()

Running Appid: 730. 0.0% Complete
	1000 reviews found
Running Appid: 1172470. 1.0% Complete
	1000 reviews found
Running Appid: 578080. 2.0% Complete
	1000 reviews found
Running Appid: 1623730. 3.0% Complete
	1000 reviews found
Running Appid: 440. 4.0% Complete
	1000 reviews found
Running Appid: 1938090. 5.0% Complete
	1000 reviews found
Running Appid: 1063730. 6.0% Complete
	1000 reviews found
Running Appid: 2358720. 7.0% Complete
	1000 reviews found
Running Appid: 271590. 8.0% Complete
	1000 reviews found
Running Appid: 550. 9.0% Complete
	1000 reviews found
Running Appid: 304930. 10.0% Complete
	1000 reviews found
Running Appid: 1599340. 11.0% Complete
	1000 reviews found
Running Appid: 236390. 12.0% Complete
	1000 reviews found
Running Appid: 2246340. 13.0% Complete
	1000 reviews found
Running Appid: 553850. 14.0% Complete
	1000 reviews found
Running Appid: 230410. 15.0% Complete
	1000 reviews found
Running Appid: 1245620. 16.0% Complete
	1000 reviews found
Running Appid: 105600. 17

In [119]:
df.head()

,appid,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,primarily_steam_deck,timestamp_dev_responded,developer_response
0,730,190365609,"{'steamid': '76561199104360494', 'num_games_ow...",english,"Counter-Strike 2: Not an Update, a Rebirth\nAt...",1742134216,1744319078,True,2917,72,0.988634109497070312,23,False,False,False,False,NaN,NaN
1,730,184994021,"{'steamid': '76561197960326460', 'num_games_ow...",english,[i] More cheaters than on Tinder [/i]\n\nOvera...,1736229250,1736229250,True,709,291,0.949078738689422607,0,True,False,False,False,NaN,NaN
2,730,185804771,"{'steamid': '76561199627926848', 'num_games_ow...",english,Its a very good game and here is the advantage...,1737143278,1737143278,True,587,170,0.941660702228546143,23,True,False,False,False,NaN,NaN
3,730,176252358,"{'steamid': '76561198444372929', 'num_games_ow...",english,A year has passed. Armory update is a cash gra...,1727914498,1727914498,False,1096,30,0.927736222743988037,0,True,False,False,False,NaN,NaN
4,730,172440169,"{'steamid': '76561198893790695', 'num_games_ow...",english,terrible anti cheat people are getting false b...,1724050740,1724305603,False,701,25,0.923591136932373047,0,False,False,False,False,NaN,NaN


In [ ]:
df.to_csv('../../data/top_1000_game_reviews.gz', index=False, compression='gzip')